In [5]:
import mlflow
import mlflow.sklearn
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
# Preprocessing 
df = pd.read_csv("datasets/insurance.csv")
df = df.drop(columns=['region'])
df['sex']=df['sex'].map({'female':0,'male':1}).astype(int)
df['smoker']=df['smoker'].map({'yes':0,'no':1}).astype(int)

In [3]:
# Training Data
X = df[["age", "bmi", "children", "smoker", "charges"]]
y = df[["sex"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
# Save the trained model locally first
lr_model = LogisticRegression().fit(X_train, y_train)
lr_model_path = "lr_model"

mlflow.sklearn.save_model(sk_model=lr_model, path=lr_model_path)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
# Create Python Class
class CustomPredict(mlflow.pyfunc.PythonModel):
    
    # Method for loading model
    def load_context(self, context):
        self.model = mlflow.sklearn.load_model("./lr_model/")
        
    # Method for custom inference    
    def predict(self, context, model_input):
        predictions = self.model.predict(model_input)
    
        decoded_predictions = []  
        for prediction in predictions:
            if prediction == 0:
                decoded_predictions.append("female")
            else:
                decoded_predictions.append("male")
        
        return decoded_predictions

In [10]:
# Log the pyfunc model 
mlflow.pyfunc.log_model(
	artifact_path="lr_pyfunc", 
  	# Set model to use CustomPredict Class
	python_model=CustomPredict(), 
	artifacts={"lr_model": "lr_model"}
)

2024/08/07 14:29:33 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


In [20]:
mlflow.pyfunc.save_model(path='custome_model', python_model=CustomPredict())

In [21]:
run = mlflow.last_active_run()
run_id = run.info.run_id

# Load the model in python_function format
loaded_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/lr_pyfunc")

In [22]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: lr_pyfunc
  flavor: mlflow.pyfunc.model
  run_id: f335d0c15bd34348822391c9652626a2

In [23]:
# loaded_model.predict(X_test.iloc[10])

In [8]:
my_pred = CustomPredict()

In [30]:
# my_pred.predict()